### Python libraries import

In [1]:
import pandas as pd
import numpy as np
from pyDOE2 import *

### Processing user input

In [2]:
# Import csv file into a dataframe
parameters = pd.read_csv('proCFPS_parameters.csv')

### Recover fixed parameters and relative maximum concentrations

In [3]:
# Recover fixed parameters
fixed_parameters = parameters[parameters['Status'] == 'fixed']
fixed_parameters = fixed_parameters['Parameter'].tolist()

In [4]:
# Recover maximum concentrations of fixed parameters
maximum_fixed_concentrations = parameters[parameters['Status'] == 'fixed']
maximum_fixed_concentrations = maximum_fixed_concentrations['Maximum concentration'].tolist()

### Recover variable parameters and relative maximum concentrations

In [7]:
# Recover variable parameters
variable_parameters = parameters[parameters['Status'] == 'variable']
variable_parameters = variable_parameters['Parameter'].tolist()
n_variable_parameters = len(variable_parameters)

In [8]:
# Recover maximum concentrations of variable parameters
maximum_variable_concentrations = parameters[parameters['Status'] == 'variable']
maximum_variable_concentrations = maximum_variable_concentrations['Maximum concentration'].to_numpy()
maximum_variable_concentrations = np.reshape(maximum_variable_concentrations, (1, n_variable_parameters))

### Latin hypercube sampling

In [9]:
# Define number of parameters to be sampled
n_parameters = n_variable_parameters

# Define number of samples to generate
n_samples = 100

# Define number of parameters' concentration ratios
n_ratios = 5

# Perform latin hypercube sampling
sampling = lhs(n_parameters, samples=n_samples)

# Define levels
levels = (1 / n_ratios)

level_1 = levels
level_2 = level_1*2
level_3 = level_1*3
level_4 = level_1*4
level_5 = level_1*5

In [10]:
# Generate multi-level list 
levels_list = []

for sample in sampling:
    new_sample = []
    
    for parameter in sample:
        
        if parameter < level_1:
            new_sample.append(0)
            continue
        
        if parameter > level_1 and parameter <= level_2:
            new_sample.append(level_1)
            continue
        
        if parameter > level_2 and parameter <= level_3:
            new_sample.append(level_2)
            continue
        
        if parameter > level_3 and parameter <= level_4:
            new_sample.append(level_3)
            continue
            
        if parameter > level_4 and parameter <= level_5:
            new_sample.append(level_4)
            continue
    
    levels_list.append(new_sample)

# Convert multi-level list to a multi-level array
levels_array = np.asarray(levels_list)

### Generating multi-level concentration array

In [11]:
# Multiply ratios array by maximum concentrations vector of variable parameters
variable_concentrations_array = np.multiply(levels_array, maximum_variable_concentrations)
variable_concentrations_array_shape = np.shape(variable_concentrations_array)
nrows_variable_concentrations_array = variable_concentrations_array_shape[0]

# Concatenate fixed concentrations arrays
GOIDNA_fixed_concentration = np.full((nrows_variable_concentrations_array, 1), 50)
GFPDNA_fixed_concentration = np.full((nrows_variable_concentrations_array, 1), 50)

final_concentrations_array = np.concatenate((variable_concentrations_array, GOIDNA_fixed_concentration), axis=1)
final_concentrations_array = np.concatenate((final_concentrations_array, GFPDNA_fixed_concentration), axis=1)
final_concentrations_df = pd.DataFrame(final_concentrations_array)

In [12]:
all_parameters = parameters['Parameter'].tolist()
output = final_concentrations_df.to_csv('initial_plate.csv', header=all_parameters)